In [1]:
import importlib
import tensorflow as tf
import pandas as pd
import mpra_model
import h5py
importlib.reload(mpra_model)
import numpy as np
from tqdm import tqdm
import sklearn
from sklearn import model_selection
import scipy.stats
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

2023-11-27 18:23:50.952529: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data_file = '../data/lenti_MPRA_embed/sei_HepG2.h5'
model_file = '../model/lenti_MPRA/lenti_MPRA_embed/HepG2/sei.h5'

In [3]:
cnn_config = {
    'activation':'exponential',
    'reduce_dim': 128,
    'conv1_filter':196,
    'conv1_kernel':7,
    'dropout1':0.2,
    'res_filter':5,
    'res_layers':3,
    'res_pool':5,
    'res_dropout':0.2,
    'conv2_filter':256,
    'conv2_kernel':7,
    'pool2_size':4,
    'dropout2':0.2,
    'dense':512,
    'dense2':256,
    'l_rate':0.0001
}

file = h5py.File(data_file,'r')
seq = file['seq'][()]
target = file['mean'][()]
x_train,x_test,y_train,y_test=model_selection.train_test_split(seq,target,random_state=42,test_size=0.1)
x_train,x_valid,y_train,y_valid = model_selection.train_test_split(x_train,y_train,random_state=42,test_size=0.1)
with tf.device("CPU"):
        trainset = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(256*4).batch(256)
        validset = tf.data.Dataset.from_tensor_slices((x_valid,y_valid)).shuffle(256*4).batch(256)
        testset = tf.data.Dataset.from_tensor_slices((x_test,y_test)).shuffle(256*4).batch(256)

model = mpra_model.rep_cnn(seq[0].shape,cnn_config)

optimizer = tf.keras.optimizers.Adam(learning_rate=cnn_config['l_rate'])
loss = tf.keras.losses.MeanSquaredError()
model.compile(optimizer=optimizer,
                loss=loss,
                metrics=['mse'])
earlyStopping_callback = tf.keras.callbacks.EarlyStopping(
        patience=10, restore_best_weights=True
    )
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.2,
        patience=5, min_lr=1e-8)
checkpoint = tf.keras.callbacks.ModelCheckpoint(
                                    model_file,
                                    monitor='val_loss',
                                    save_best_only=True,
                                    mode = 'min',
                                    save_freq='epoch',)
model.fit(
        trainset,
        epochs=100,
        batch_size=512,
        shuffle=True,
        validation_data = validset,
        callbacks=[earlyStopping_callback,reduce_lr
                   ,checkpoint]
    )
pred_y = []
y_test = []
for i,(x,y) in enumerate(testset):
    pred_y.extend(model.predict(x))
    y_test.extend(y)

print(scipy.stats.pearsonr(np.squeeze(pred_y),np.squeeze(y_test)))

2023-11-27 18:23:56.540902: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78973 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:07:00.0, compute capability: 8.0
/home/ztang/.conda/envs/jax_tf/lib/python3.9/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Epoch 1/100


2023-11-27 18:24:01.721400: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [113300,960,16]
	 [[{{node Placeholder/_0}}]]
2023-11-27 18:24:01.721619: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [113300]
	 [[{{node Placeholder/_1}}]]
2023-11-27 18:24:03.654953: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8800
2023-11-27 18:24:04.303167: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2

443/443 [==============================] - ETA: 0s - loss: 0.3283 - mse: 0.3283

2023-11-27 18:24:17.471301: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [12589,960,16]
	 [[{{node Placeholder/_0}}]]
2023-11-27 18:24:17.471523: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [12589]
	 [[{{node Placeholder/_1}}]]


443/443 [==============================] - 17s 27ms/step - loss: 0.3283 - mse: 0.3283 - val_loss: 0.6618 - val_mse: 0.6618 - lr: 1.0000e-04
Epoch 2/100
443/443 [==============================] - 11s 24ms/step - loss: 0.2942 - mse: 0.2942 - val_loss: 0.3510 - val_mse: 0.3510 - lr: 1.0000e-04
Epoch 3/100
443/443 [==============================] - 10s 24ms/step - loss: 0.2863 - mse: 0.2863 - val_loss: 0.3578 - val_mse: 0.3578 - lr: 1.0000e-04
Epoch 4/100
443/443 [==============================] - 11s 24ms/step - loss: 0.2821 - mse: 0.2821 - val_loss: 0.3055 - val_mse: 0.3055 - lr: 1.0000e-04
Epoch 5/100
443/443 [==============================] - 11s 24ms/step - loss: 0.2783 - mse: 0.2783 - val_loss: 0.2988 - val_mse: 0.2988 - lr: 1.0000e-04
Epoch 6/100
443/443 [==============================] - 11s 24ms/step - loss: 0.2753 - mse: 0.2753 - val_loss: 0.3059 - val_mse: 0.3059 - lr: 1.0000e-04
Epoch 7/100
443/443 [==============================] - 11s 24ms/step - loss: 0.2722 - mse: 0.2722 - 

2023-11-27 18:30:10.760018: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [13988]
	 [[{{node Placeholder/_1}}]]
2023-11-27 18:30:10.760246: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [13988]
	 [[{{node Placeholder/_1}}]]


6/6 [==============================] - 0s 8ms/step
PearsonRResult(statistic=0.7527342712209032, pvalue=0.0)
